In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import pyarrow.parquet as pq
# import pyarrow as pa

# # Открытие первого файла Parquet
# table1 = pq.read_table('./Data/geo_train.parquet')

# # Открытие второго файла Parquet
# table2 = pq.read_table('./Data/geo_test.parquet')

# # Конкатенация таблиц
# table_concatenated = pa.concat_tables([table1, table2])

# # Сохранение результата в новый файл Parquet
# pq.write_table(table_concatenated, './Data/geo.parquet')

In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T


data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

24/06/15 13:55:32 WARN Utils: Your hostname, dimk-B450M-DS3H resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface eno1)
24/06/15 13:55:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/15 13:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/15 13:55:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.driver.memoryOverhead', '4g'),
 ('spark.app.id', 'local-1718448933090'),
 ('spark.app.startTime', '1718448932610'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirect

In [4]:
geo_train = spark.read.options(header=True, inferSchema=True).parquet("./Data/geo_train.parquet")
geo_train.show(2)

+--------------------+--------------------+---------+---------+---------+
|           client_id|          event_time|geohash_4|geohash_5|geohash_6|
+--------------------+--------------------+---------+---------+---------+
|6590fef3f66b31fcf...|2022-11-04 13:03:...|    24259|    93918|   962803|
|6590fef3f66b31fcf...|2022-11-09 12:52:...|    24259|    93918|   962803|
+--------------------+--------------------+---------+---------+---------+
only showing top 2 rows



In [5]:
geo_test = spark.read.options(header=True, inferSchema=True).parquet("./Data/geo_test.parquet")
geo_test.show(2)

+--------------------+--------------------+---------+---------+---------+
|           client_id|          event_time|geohash_4|geohash_5|geohash_6|
+--------------------+--------------------+---------+---------+---------+
|0d5c0c50cccc7e786...|2022-01-25 08:47:...|    41966|    54997|  1976570|
|0d5c0c50cccc7e786...|2022-01-29 09:02:...|    41966|    54997|  1976570|
+--------------------+--------------------+---------+---------+---------+
only showing top 2 rows



In [6]:
geo_df = spark.read.options(header=True, inferSchema=True).parquet("./Data/geo.parquet")
geo_df.show(2)

+--------------------+--------------------+---------+---------+---------+
|           client_id|          event_time|geohash_4|geohash_5|geohash_6|
+--------------------+--------------------+---------+---------+---------+
|309c0e909835757db...|2022-08-27 09:56:...|    39879|   144891|  1959174|
|309c0e909835757db...|2022-08-14 07:13:...|    39879|   144891|  1959174|
+--------------------+--------------------+---------+---------+---------+
only showing top 2 rows



In [7]:
from ptls.preprocessing import PysparkDataPreprocessor

preprocessor = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=["geohash_4",
                   "geohash_5",
                   "geohash_6",
                   ],
)


In [8]:
train_dataset = preprocessor.fit_transform(geo_train).persist()
train_dataset.count()

24/06/15 13:55:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 13:55:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 13:55:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 13:55:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 13:55:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 13:55:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 1

622304

In [9]:
train_dataset.show(2)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           client_id|          event_time|           geohash_4|           geohash_5|           geohash_6|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|0002f001b84dcf00b...|[1640980191, 1641...|[88, 88, 88, 88, ...|[1511, 1995, 351,...|[8893, 7753, 3760...|
|0023bd56d71ad384d...|[1641184989, 1641...|[38, 38, 38, 38, ...|[907, 907, 907, 9...|[10001, 10001, 10...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [10]:
test_dataset = preprocessor.fit_transform(geo_test).persist()
test_dataset.count()

24/06/15 14:23:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:23:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:23:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:23:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:23:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:23:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 1

166747

In [11]:
df_dataset = preprocessor.fit_transform(geo_df).persist()
df_dataset.count()

24/06/15 14:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:31:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:31:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 14:31:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/15 1

789051

In [1]:
train_dataset.write.parquet('./Data/train_geo_preprocessor.parquet', mode='overwrite')
test_dataset.write.parquet('./Data/test_geo_preprocessor.parquet', mode='overwrite')
df_dataset.write.parquet('./Data/df_geo_preprocessor.parquet', mode='overwrite')

NameError: name 'train_dataset' is not defined